In [1]:
!pip install neattext

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 2.3 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import neattext.functions as nfx
import plotly.express as plx
from sklearn.metrics import classification_report
import keras
from keras.layers import Embedding,Dense,LSTM,GlobalMaxPooling1D,Input
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
from keras.models import Sequential
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm

In [3]:
data=pd.read_csv('fake reviews dataset.csv')
data.head()

,category,rating,label,review_text
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor..."
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I..."
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i..."
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...


In [6]:
# Assuming the column containing the class labels is named 'label'
# based on the information from the global variables.
data['label'].value_counts()

,count
label,
CG,20216
OR,20216


In [8]:
data['label'].value_counts().index.values

array(['CG', 'OR'], dtype=object)

In [10]:
train_data,test_data=train_test_split(data,test_size=0.2,random_state=10)
train_data['label'].value_counts().index.values

array(['CG', 'OR'], dtype=object)

In [11]:
import plotly.express as plx

# Assuming 'Suicide' and 'Not Suicide' are the actual values in the 'class' column
class_counts = train_data['label'].value_counts()

# Create a bar chart using plotly.express with the correct arguments
fig = plx.bar(
    x=class_counts.index,  # x-axis: unique values from the 'class' column
    y=class_counts.values,  # y-axis: counts of each unique value
    color=class_counts.index,  # color bars based on class
    labels={'x': 'Class', 'y': 'Count'},  # set axis labels for clarity
    title='Distribution of Classes in Training Data'
)

fig.show()

In [13]:
def clean_text(text):
    text_length=[]
    cleaned_text=[]
    for sent in tqdm(text):
        sent=sent.lower()
        sent=nfx.remove_special_characters(sent)
        sent=nfx.remove_stopwords(sent)
        text_length.append(len(sent.split()))
        cleaned_text.append(sent)
    return cleaned_text,text_length
# Assuming the text data is stored in the column named 'category'
# Replace 'category' with the actual name of the column containing the text data.
cleaned_train_text,train_text_length=clean_text(train_data['category'])
cleaned_test_text,test_text_length=clean_text(test_data['category'])

100%|██████████| 8087/8087 [00:00<00:00, 213014.33it/s]


In [46]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Assuming your data is in a CSV file named 'your_data.csv'
# Replace 'your_data.csv' with the actual file name.
df = pd.read_csv('fake reviews dataset.csv')  # Load your data into a pandas DataFrame

# Convert labels ('CG' = Genuine, 'OR' = Fake) to binary (1 for genuine, 0 for fake)
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Tokenize and preprocess the review texts
max_words = 10000  # Limit the number of words to consider in the tokenizer
max_len = 100      # Maximum length of review sequence

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['review_text'])

# Convert reviews to sequences and pad them
X = tokenizer.texts_to_sequences(df['review_text'])
X = pad_sequences(X, maxlen=max_len)

# Target variable
y = df['label']

# Split dataset into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [47]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Define the model
model = Sequential()
model.add(Embedding(max_words, 128, input_length=max_len))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.2)


Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning:

Argument `input_length` is deprecated. Just remove it.



405/405 ━━━━━━━━━━━━━━━━━━━━ 70s 166ms/step - accuracy: 0.7877 - loss: 0.4214 - val_accuracy: 0.9266 - val_loss: 0.1860
Epoch 2/5
405/405 ━━━━━━━━━━━━━━━━━━━━ 67s 165ms/step - accuracy: 0.9374 - loss: 0.1711 - val_accuracy: 0.9201 - val_loss: 0.2145
Epoch 3/5
405/405 ━━━━━━━━━━━━━━━━━━━━ 80s 159ms/step - accuracy: 0.9554 - loss: 0.1264 - val_accuracy: 0.9358 - val_loss: 0.1747
Epoch 4/5
405/405 ━━━━━━━━━━━━━━━━━━━━ 66s 163ms/step - accuracy: 0.9671 - loss: 0.0905 - val_accuracy: 0.9329 - val_loss: 0.1825
Epoch 5/5
405/405 ━━━━━━━━━━━━━━━━━━━━ 85s 170ms/step - accuracy: 0.9743 - loss: 0.0705 - val_accuracy: 0.9340 - val_loss: 0.1951


In [49]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (None, 100, 128)            │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,988,421 (15.21 MB)

 Trainable params: 1,329,473 (5.07 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,658,948 (10.14 MB)

In [48]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


253/253 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.9363 - loss: 0.1779
Test Accuracy: 93.31%


In [54]:
import numpy as np
from sklearn.metrics import classification_report

print('TESTING DATA CLASSIFICATION REPORT \n \n')
# Use predict to get probabilities and then argmax for class labels
y_pred_test = np.argmax(model.predict(test_text_pad), axis=-1)
print(classification_report(test_output, y_pred_test,
                            target_names=lbl_target.inverse_transform([0,1])))

print('TRAINING DATA CLASSIFICATION REPORT \n \n')
# Use predict to get probabilities and then argmax for class labels
y_pred_train = np.argmax(model.predict(train_text_pad), axis=-1)
print(classification_report(train_output, y_pred_train,
                           target_names=lbl_target.inverse_transform([0,1])))

TESTING DATA CLASSIFICATION REPORT 
 

253/253 ━━━━━━━━━━━━━━━━━━━━ 8s 28ms/step
              precision    recall  f1-score   support

          CG       0.49      1.00      0.66      3961
          OR       0.00      0.00      0.00      4126

    accuracy                           0.49      8087
   macro avg       0.24      0.50      0.33      8087
weighted avg       0.24      0.49      0.32      8087

TRAINING DATA CLASSIFICATION REPORT 
 

   4/1011 ━━━━━━━━━━━━━━━━━━━━ 19s 19ms/step

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



1011/1011 ━━━━━━━━━━━━━━━━━━━━ 22s 22ms/step
              precision    recall  f1-score   support

          CG       0.50      1.00      0.67     16255
          OR       0.00      0.00      0.00     16090

    accuracy                           0.50     32345
   macro avg       0.25      0.50      0.33     32345
weighted avg       0.25      0.50      0.34     32345



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [58]:
twt = ['it is nice bowl']
twt = tokenizer.texts_to_sequences(twt)
twt = pad_sequences(twt, maxlen=50)

prediction = model.predict(twt)[0][0]
print(prediction)

if(prediction > 0.90):
    print("original review")
else:
    print("fake review")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
0.9723289
original review


In [61]:
twt = ['alright for the price,but was a little disappointed with the quality of the product']
twt = tokenizer.texts_to_sequences(twt)
twt = pad_sequences(twt, maxlen=50)


prediction = model.predict(twt)[0][0]
print(prediction)

if(prediction > 0.90):
    print("original review")
else:
    print("fake review")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
0.0074966853
fake review
